In [1]:
import leuvenmapmatching as mm
from leuvenmapmatching.map.inmem import InMemMap
from leuvenmapmatching.matcher.distance import DistanceMatcher
import pickle
import json
import numpy as np
import networkx as nx
import osmnx as ox
import tqdm, os
from collections import Counter, defaultdict

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [12]:
import pandas as pd

df_recd = pd.read_pickle(
    open('../dataset/records_4w_tracklet.pkl', 'rb'))
df_recd = pd.DataFrame.from_dict(df_recd, orient='columns')

In [13]:
tklet_path = '../dataset/sim_traj/tracklet.json'
import json

tklet = json.load(open(tklet_path, 'rb'))

image_car_mapping = {}

# 遍历data字典，提取image_name和car_plate的对应关系
for key, value in tklet.items():
    image_name = value["image_name"]
    car_plate = value["car_plate"]
    image_car_mapping[image_name] = car_plate

In [14]:
data_file = map_file = "../dataset/"
f = open(data_file + "track_carplate.pkl", "rb")
carplate = pickle.load(f)

In [15]:
# carplate

In [16]:
df_recd['plate_feature'] = list(map(lambda x: carplate[0][x] if x in carplate[0].keys() else None, df_recd['image_name'].values))

In [17]:
df_recd['plate_text'] = list(map(lambda x: carplate[1][x] if x in carplate[1].keys() else None, df_recd['image_name'].values))

In [18]:
df_recd['time'] = df_recd['time'].values.astype(float)

In [19]:
df_recd[df_recd.vehicle_id == 6005]

,vehicle_id,time,image_name,camera_id,car_feature,plate_feature,plate_text,tklet_nodes
34772,6005,115.485550,image_161_0709_01.jpg,104,"[0.04503826, -0.002611258, -0.10928233, -0.014...",None,None,[104]
34773,6005,119.845650,image_174_0709_01.jpg,221,"[0.00079358084, 0.012050003, -0.08467655, -0.0...",None,None,"[93, 221, 443]"
34774,6005,115.325546,image_159_0709_01.jpg,104,"[0.03690407, -0.013000028, -0.020414937, -0.03...",None,None,[104]
34775,6005,118.045609,image_170_0709_01.jpg,502,"[0.026666282, 0.023424711, -0.048881248, -0.01...",None,None,"[1581, 502, 93]"
34776,6005,123.925743,image_176_0709_01.jpg,222,"[0.04490466, 0.031877365, -0.017897364, 0.0030...","[-0.036121022, 0.045402184, -0.023910783, 0.02...",云H4J2Y4,"[1488, 222, 1041]"
34777,6005,117.885605,image_168_0709_01.jpg,502,"[0.010126831, 0.048048392, -0.05787726, -0.067...",None,None,"[1581, 502, 93]"
34778,6005,119.965652,image_175_0709_01.jpg,221,"[0.03593946, 0.0022448231, 0.015212699, -0.012...",None,None,"[93, 221, 443]"
34779,6005,115.125542,image_157_0709_01.jpg,104,"[0.07081916, 0.02596825, -0.009538708, 0.00195...","[-0.049215246, 0.09163502, -0.00047773123, 0.0...",云H4J2Y4,[104]
34780,6005,117.965607,image_169_0709_01.jpg,502,"[0.014234365, 0.050493922, -0.06255925, -0.054...",None,None,"[1581, 502, 93]"
34781,6005,117.605598,image_165_0709_01.jpg,502,"[0.04501919, 0.05290262, -0.025966564, -0.0293...","[-0.024320705, 0.052298494, -0.003094544, 0.02...",云H4J2Y4,"[1581, 502, 93]"


In [20]:
df_recd['true_plate'] = df_recd.image_name.apply(lambda x: image_car_mapping[x])
df_recd[df_recd.vehicle_id == 6005]

,vehicle_id,time,image_name,camera_id,car_feature,plate_feature,plate_text,tklet_nodes,true_plate
34772,6005,115.485550,image_161_0709_01.jpg,104,"[0.04503826, -0.002611258, -0.10928233, -0.014...",None,None,[104],云H4J2Y4
34773,6005,119.845650,image_174_0709_01.jpg,221,"[0.00079358084, 0.012050003, -0.08467655, -0.0...",None,None,"[93, 221, 443]",云H4J2Y4
34774,6005,115.325546,image_159_0709_01.jpg,104,"[0.03690407, -0.013000028, -0.020414937, -0.03...",None,None,[104],云H4J2Y4
34775,6005,118.045609,image_170_0709_01.jpg,502,"[0.026666282, 0.023424711, -0.048881248, -0.01...",None,None,"[1581, 502, 93]",云H4J2Y4
34776,6005,123.925743,image_176_0709_01.jpg,222,"[0.04490466, 0.031877365, -0.017897364, 0.0030...","[-0.036121022, 0.045402184, -0.023910783, 0.02...",云H4J2Y4,"[1488, 222, 1041]",云H4J2Y4
34777,6005,117.885605,image_168_0709_01.jpg,502,"[0.010126831, 0.048048392, -0.05787726, -0.067...",None,None,"[1581, 502, 93]",云H4J2Y4
34778,6005,119.965652,image_175_0709_01.jpg,221,"[0.03593946, 0.0022448231, 0.015212699, -0.012...",None,None,"[93, 221, 443]",云H4J2Y4
34779,6005,115.125542,image_157_0709_01.jpg,104,"[0.07081916, 0.02596825, -0.009538708, 0.00195...","[-0.049215246, 0.09163502, -0.00047773123, 0.0...",云H4J2Y4,[104],云H4J2Y4
34780,6005,117.965607,image_169_0709_01.jpg,502,"[0.014234365, 0.050493922, -0.06255925, -0.054...",None,None,"[1581, 502, 93]",云H4J2Y4
34781,6005,117.605598,image_165_0709_01.jpg,502,"[0.04501919, 0.05290262, -0.025966564, -0.0293...","[-0.024320705, 0.052298494, -0.003094544, 0.02...",云H4J2Y4,"[1581, 502, 93]",云H4J2Y4


In [11]:
df_recd = df_recd.sort_values(['vehicle_id', 'time'])
df_recd['Group'] = (df_recd['camera_id'] != df_recd['camera_id'].shift()).cumsum()

In [23]:
sum(df_recd.plate_text.values == df_recd.true_plate.values) / df_recd.plate_text.dropna().shape[0]

0.6271159996879632

In [12]:
df_ans = df_recd.dropna().groupby('Group').agg(lambda x: x.values[0])
df_recd = df_recd.groupby('Group').agg(lambda x: x.values[0])
df_recd.update(df_ans)
float_cols = df_recd.select_dtypes(include=['float']).columns
df_recd[float_cols] = df_recd[float_cols].astype(int)
df_recd = df_recd.reset_index()
df_recd['record_id'] = df_recd['Group'] - 1
df_recd = df_recd.drop(columns=['Group'])

In [13]:
# df_recd[df_recd.vehicle_id == 6005]
df_recd[df_recd.record_id == 335]


,vehicle_id,time,image_name,camera_id,car_feature,plate_feature,plate_text,tklet_nodes,record_id
335,81,472,image_1975_0710_004.jpg,741,"[-0.044881143, -0.00403498, -0.1328665, -0.065...","[0.040428832, -0.005888003, -0.09492472, 0.025...",京B435EF,[741],335


In [14]:
pickle.dump(df_recd.to_dict('records'), open(data_file + 'records_4w.pkl', 'wb'))

In [15]:
df_recd.shape

(18108, 9)